# 암호 응용 및 실습 HW2 (총점: 70점)

**실습 과제 코드 작성시 유의사항**

---
1. 과제 제출 시에는 아래 채워야 하는 부분에 코드만 작성해서 제출. 과제 수행 시에 중간 결과 확인을 위해 연습 할 때 수행했던 코드들 및 새롭게 생성한 코드 블록들은 모두 삭제해서 제출해야 함. 
2. 각 문항마다 평가 코드 부분이 포함되어 있음. 실제 과제 평가 시에 활용될 부분이니 참고해서 과제를 수행할 수 있으나, 과제 제출 시에 해당 부분을 수정 및 삭제하지 않고 그대로 제출해야 함.
3. 과제 수행 시에 복사 본을 만들어서 활용하고, 제출 파일은 필요한 부분만 작성해서 제출할 것을 권장함

---





## pycryptodome 설치

In [ ]:
!pip install pycryptodome

# 1. 거듭 제곱 알고리즘 [10점]
11주차 공개키 암호 강의자료의 p.23 거듭제곱 알고리즘의 의사코드를 참고하여, 

1. naive 한 모듈로 거듭제곱 계산 함수 
2. 빠른 모듈로 거듭제곱 알고리즘 함수 

위의 두 함수를 직접 구현하고, 해당 함수의 출력으로 **거듭제곱한 결과 값 (y)** 뿐만 아니라 **필요한 곱셈의 수 (m)**를 계산하여 함께 return 하도록 작성하여라.

- 입력: x 밑, e 지수, n 모듈로
- 출력: y = x**e (mod n), m 필요한 곱셈 수
- 예를 들어, 2**3 (mod 5)을 계산하기 위해서는 총 2번의 곱셈이 필요하다. 이 때, y = 3, m=2 가 된다.


In [12]:
def expModNaive(x,e,n):
  y=x
  if(e==0):
    return 1, 1 # e가 0일 경우에는 1,1을 리턴하게 한다
  else:
    m=e-1
  for i in range(m):
    y=y*x % n
  return y, m

def expModFast(x,e,n):
  eb=bin(e)#이진수로 변환한다
  eb=eb[2:]
  y=x
  m=len(eb)
  h=1
  for i in range(m-2,-1,-1):
    y=y*y%n
    if eb[i]=='1':
      h = h*y % n
  if eb[m-1]=="1":#0,1 을 담당하는 비트열이 1일 경우, x를 곱해준다.
    h= h*x%n
  
  return h, m


## 평가 코드

In [17]:
print(expModNaive(2,3,5))
print(expModFast(2,65536,5))

(3, 2)
(1, 17)


# 2. 해시 함수 무결성 검증 with AES [20점]
주어진 text 파일을 AES 로 암호화 한 후에 이를 복호화 한 파일을 가지고 원래 파일과 복호화된 파일 간의 무결성 검증을 하는 코드를 작성하여라

**참고사항**
- AES 암호화 및 복호화 시에 CBC_MODE 를 사용하고, 적법한 메세지 패딩 수행 (적법한 패딩이면, 어떠한 패딩을 사용하든지 상관 없음)
- getFileHash 함수와 myEAS 클래스를 활용하여 AEShashCheck 함수를 통해 원래 파일과 복호화된 파일 간의 무결성 검증 수행

In [ ]:
from Crypto.Hash import SHA256 as SHA
from Crypto.Cipher import AES

In [ ]:

def getFileHash(filename):
  """
  file 경로가 주어지면 해당 파일에 대한 SHA256 해시 값을 계산
  """
  with open(filename,"rb") as f:
    a=f.read().decode()
  sha=SHA.new()
  sha.update(a.encode())
  hashval = sha.hexdigest()#16진수. 64비트

  return hashval

class myAES():
  """
  메세지 채움 및 후처리, 암호화 및 복호화 함수를 포함한 AES class 정의
  (대칭키 암호 실습 코드 참고)
  <차이점> 
  1. 암호화 및 복호화 시에 직접 평문 및 암호문을 입력 및 출력으로 하지 않고, 
  파일로 읽어오고 저장함
  2. 따라서 암호화 및 복호화 함수의 return 값이 없이 파일로 저장됨
  """
  def __init__(self,keytext, ivtext, keysize = 128):
    # 키 생성 및 초기 값 생성 코드 작성
    hash_fn=SHA.new()
    hash_fn.update(keytext.encode())
    key=hash_fn.digest()
    self.key=key[:16]

    hash_fn.update(ivtext.encode())
    iv=hash_fn.digest()
    self.iv=iv[:16]

  def makeEnabled(self, plaintext):
    # 메세지 패딩 코드 작성
    plaintext=plaintext.decode()
    fillersize = 0
    textsize = len(plaintext)
    if textsize %16 != 0: 
      fillersize = 16-textsize%16
    filler = '0'*fillersize
    header = '%d' %(fillersize)
    gap = 16-len(header)
    header += '#'*gap
    filler_msg=header+plaintext+filler
    return filler_msg

  def postDec(self, dec_msg):
    # 복호화 후 메세지 후처리 코드 작성 (패딩 제거)

    header = dec_msg[:16].decode()#[:16] 안써도 틀리진 않지만, 쓰면 더 명확하다
    fillersize = int(header.split('#')[0])
    if fillersize !=0:
      decmsg = dec_msg[16:-fillersize].decode()
    else:
      decmsg = dec_msg[16:].decode()

    return decmsg

  def encrypt(self, inputfile, outputfile):
    #inputfile: 암호화 할 메세지가 담겨있는 입력 파일 경로
    #outputfile: 암호화 후 암호문을 저장할 출력 파일 경로
    with open(inputfile,'rb') as f:
      plaintext = f.read()
    # 패딩 및 암호화 수행 코드 작성

    plaintext=self.makeEnabled(plaintext)
    aes=AES.new(self.key,AES.MODE_CBC, self.iv)
    encmsg=aes.encrypt(plaintext.encode())
    # 암호문 파일 저장 코드 작성
    with open(outputfile, 'wb') as f:
      f.write(encmsg)#암호화해서 byte형태

    # return 값 없음
  def decrypt(self, inputfile, outputfile):
    #inputfile: 암호문이 담겨있는 입력 파일 경로
    #outputfile: 복호화 후 메세지을 저장할 출력 파일 경로
    with open(inputfile, 'rb') as f:
      ciphertext = f.read()
    
    # 복호화 및 후처리 수행 코드 작성
    aes = AES.new(self.key, AES.MODE_CBC, self.iv)
    decmsg = aes.decrypt(ciphertext)

    decmsg=self.postDec(decmsg)
    
    with open(outputfile, 'wb') as f:
      f.write(decmsg.encode())
    # return 값 없음

def AEShashCheck(file1, file2, file3, aes_params):
  """
  <함수 설명>
  원래 메세지 파일인 file1 을 AES 암호화 하여 file2에 저장
  file2 의 암호문을 복호화 하여 file3 에 저장
  file1과 file3의 해시 값 비교를 통해서 암복호화 후의 메세지의 무결성 검증
  <AES 관련 입력 설명>
  - AES 암호는 위에 정의된 myAES 클래스를 활용
  - aes_params 는 dictionary 타입으로 키로 'keytext', 'ivtext', 'keysize' 를 지님 
  """
  msg="hello"
  with open(file1, 'wb') as f:
    f.write(msg.encode())
  myaes = myAES(aes_params['keytext'], aes_params['ivtext'], aes_params['keysize'])
  myaes.encrypt(file1, file2)
  myaes.decrypt(file2, file3)

  # file1 과 file3 의 해시 값을 비교하는 코드 작성
  # 두 파일의 해시 값이 같을 경우에는 "Two files are the same"
  # 두 파일의 해시 값이 다를 경우에는 "The files are different" 가 출력됨
 
  hashval1 = getFileHash(file1)
  hashval2 = getFileHash(file3)

  if hashval1 == hashval2:
    print("Two files are the same.")
  else:
    print("Two files are different.")


## 평가 코드

In [ ]:
msgfile = "plaintext.txt"
cipherfile = "ciphertext.txt"
decfile = "decrypted.txt"
aes_params = {}
aes_params['keytext'] = 'secret'
aes_params['ivtext'] = 'init'
aes_params['keysize'] = 128
AEShashCheck(msgfile, cipherfile, decfile, aes_params)

Two files are the same.


# 3. 안전한 통신을 위한 RSA 사용 with 무결성 검증 [40점]
EAX_MODE 대신에 CBC_MODE 를 사용하면서 전송 시에 메세지의 해시 값을 함께 보내 무결성 검증을 만족할 경우에만 복호화를 수행하는 코드를 작성하여라

**참고사항**
- 공개키 암호 실습 수업에서는 메세지의 무결성 검정과 암호화를 동시에 수행하기 위해서 AES 암호화 시에 EAX_MODE를 사용하였음
- CBC_MODE 사용 시에는 메세지의 무결성 검증은 안 되므로 메세지의 해시 값을 통해서 무결성 검증을 위한 부분을 따로 작성해야 함
- 코드 작성시 "14. 공개키 암호 실습.ipynb" 의 "안전한 통신을 위한 RSA 사용" 부분 참고


In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256 as SHA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode, b64decode
import json

In [ ]:
def makeEncMsg(publicfile,msgfile,encfile):
  """
  publicfile: 공개키 파일
  msgfile: 암호화해서 전송할 메세지 파일
  encfile: 메세지 암호문, 암호화된 키, 무결성 정보를 포함하는 전송될 파일
  """
  # session key를 생성하고 publicfile 의 public key로 암호화 부분 코드 작성
  session_key = get_random_bytes(16)

  loaded_key=RSA.import_key(open(publicfile).read())
  cipher_rsa=PKCS1_OAEP.new(loaded_key)#암호화 클래스 만듦
  enc_session_key=cipher_rsa.encrypt(session_key)#세션키를 공개키로 암호화

  # session key 를 이용하여 원래의 msg를 암호화하는 코드 작성
  # 1. msg 읽어오기
  with open(msgfile,'rb') as f:
    msg=f.read()
  # 2. AES class 생성 및 읽어온 msg 패딩하기 (Crypto 패키지의 pad 사용하기)
  cipher_aes = AES.new(session_key, AES.MODE_CBC)#cbc모드로 태그는 나오지 않는다.
  padded_msg = pad(msg, AES.block_size)

  # 3. AES로 메세지 암호화

  ciphertext = cipher_aes.encrypt(padded_msg)

  # SHA256 해시 함수를 이용해서 메세지의 해시 값 구함
  hash_fn = SHA.new()
  hash_fn.update(msg)
  hash_msg=hash_fn.digest()
  
  # 암호화된 session key, CBC_MODE 의 initial value, 메세지 해시값, 암호화된 메세지가 포함된 
  # base64로 encode 되어 있는 encdata 정의 (공개키 암호 실습 코드 참고)
  encdata = {}
  encdata['esk'] = b64encode(enc_session_key).decode()
  encdata['iv'] = b64encode(cipher_aes.iv).decode()
  encdata['hashval'] = b64encode(hash_msg).decode()
  encdata['ciphertext'] = b64encode(ciphertext).decode()

  # json 을 이용해서 encdata 를 encfile 에 쓰기
  with open(encfile,'w') as f:
    json.dump(encdata,f)
  

def decMsg(encfile,privatefile, secret_code = None):
  """
  암호화된 메세지 및 해시 정보를 포함한 encfile 을 받았을 때에 msg 복호화 및 무결성 검사
  """
  # encfile 읽어오기 및 json을 이용해서 불러와서 b64decode 수행 부분 작성
  with open(encfile,"r") as f:
    enc_data = json.load(f)
  
  dec_data = {}
  for k in enc_data:
    dec_data[k] = b64decode(enc_data[k])
  secret_code = "TopSecret"
  
  # RSA 개인키 이용해서 AES session key 복원 부분 작성
  private_key = RSA.import_key(open('privateKey.pem').read(),secret_code)
  cipher_rsa = PKCS1_OAEP.new(private_key)
  session_key = cipher_rsa.decrypt(dec_data['esk'])

  # session key와 dec_data['iv'] 사용해서 ciphertext 복호화 부분 작성
  cipher_aes=AES.new(session_key,AES.MODE_CBC,dec_data['iv'])
  msg=cipher_aes.decrypt(dec_data['ciphertext'])

  # 메세지의 무결성 검증 (복호화한 메세지 msg 의 해시 값과 encfile 에 있던 해시 값이 같은지 검증)
  # 만약 다르면, "Sorry, the message is modified!"
  # 같으면, "The message is: "+ msg 값이 출력 되도록 함
  msg = unpad(msg, AES.block_size)
  
  hash_fn = SHA.new()
  hash_fn.update(msg)
  enc_hash=hash_fn.digest()

  
  
  if enc_hash == dec_data['hashval']:
    print("The message is "+ msg.decode())
  else:
    print("Sorry, the message is modified!")


## 평가 코드

In [ ]:
secret_code = "TopSecret"
private_key = RSA.generate(2048)
with open('privateKey.pem','wb') as f:
  f.write(private_key.exportKey(format = 'PEM',passphrase = secret_code))

public_key = private_key.publickey()
with open('PublicKey.pem', 'wb') as f:
  f.write(public_key.exportKey('PEM'))

In [ ]:
msg="hello"
with open("plaintext.txt",'wb') as f:
  f.write(msg.encode())
#이 밑은 교수님 코드 이 위에는 내가 임의로 만든 파일 코드
makeEncMsg("PublicKey.pem","plaintext.txt","encFile.bin")
with open("encFile.bin",'r') as f:
  encData = json.load(f)

for k in encData:
  print(k,": ", encData[k])

esk :  stLwqEenTaGUFQAG2PQJnKGlUgJmWUuiJ0072+E1SIMeYMh99xFjQyXvtSxa5svmx+bw6V4zK5ufYLiwrKlUId2TJ/2lvZPeMcwLfnYZEtL3eKerejXDlhjxUZZpvtmsUhmvx7Oodn1d/wD0LxpQYrWp6uIVxyxmwyK2YY726cfCfkuPmwsq80BKvxilDKxkMOQjLe80Zz5n4HiDWfxs3pQFA7XZZr8YDpyW561/RZIBLkmgCzuMNZvcH1zsM8l5CHiT5+R/MpcM+w+KTq3Us5wGe7qtjnxZvgbBtVAD58PLrsvo4f7/XISe23radAEimFFuyGCYySrHUdzR8PTZaQ==
iv :  jAv4wc8VO/YBoANgeuaXfA==
hashval :  LPJNul+wow4m6DsqxbninhsWHlwfp0JecwQzYpOLmCQ=
ciphertext :  MSVf1NEr6fIsSN3XkteUuw==


In [ ]:
decMsg("encFile.bin","privateKey.pem","TopSecret")

The message is hello


# 협업 명시
- 과제 수행 시에 함께 서로 도와가며 수행한 부분이 있다면, 반드시 아래에 명시해야 함
- 명시하지 않거나 다른 사람의 코드를 그대로 복사하였을 경우에는 copy 로 점수 없음